### This script creates a map of a cloud band and shows the properties: orientation, centroid and ellipse around the cloud band (blob)

#### Importation of appropriate packages

In [1]:
import os
import warnings
import datetime as dt

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Ellipse
from skimage import measure

import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
LON_FORMAT = LongitudeFormatter(zero_direction_label=True, degree_symbol="")
LAT_FORMAT = LatitudeFormatter(degree_symbol="")
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from cloudbandpy.io_utilities import load_ymlfile, add_startend_datetime2config
from cloudbandpy.load_driver import run_load_data
from cloudbandpy.cb_detection import detection_workflow
from cloudbandpy.figure_tools import set_fontsize
from cloudbandpy.misc import wrapTo180

In [2]:
config_file = "../config/config_cbworkflow_southPacific.yml"
config = load_ymlfile(config_file, isconfigfile=True)
config_event = config.copy()
# Changing configs to adapt to this particular example
config_event["startdate"] = "20160418.00"
config_event["enddate"] = "20160418.00"
# update datetime dates from config file
add_startend_datetime2config(config_event)
dirdata = "../data/"
config_event["clouddata_path"] = os.path.join("../data")
config_event["parameters_file"] = os.path.join(
    "../parameters/parameters_southhemisphere.yml"
)
# No need to save any files
config_event["saved_dirpath"] = ""
config_event["save_dailyvar"] = False
config_event["save_listcloudbands"] = False

2023-11-23 16:17:39,351 - io_utilities.load_ymlfile - INFO: Configuration file loaded


In [3]:
olr2process, parameters, listofdates, lats, lons, resolution = run_load_data(
    config_event
)

2023-11-23 16:17:39,359 - load_driver.run_load_data - INFO: Loading data and parameters 
2023-11-23 16:17:39,363 - io_utilities.load_ymlfile - INFO: Parameters file loaded
2023-11-23 16:17:39,364 - io_utilities.load_dataset - INFO: Loading dataset from ../data
2023-11-23 16:17:39,364 - io_utilities.load_dataset - INFO: Loading 2016 --> 2016


In [ ]:
(
    _,
    _,
    _,
    _,
    cloud_bands_over_time,
    _,
) = detection_workflow(
    var2process=olr2process,
    parameters=parameters,
    latitudes=lats,
    longitudes=lons,
    resolution=resolution,
    listofdates=listofdates,
    config=config,
)

In [ ]:
print(lats.shape)
print(lons.shape)
print(cloud_bands_over_time.shape)

In [ ]:
cloud_bands = np.copy(cloud_bands_over_time[0])


In [ ]:
blobs_regionprops = measure.regionprops(cloud_bands)

In [ ]:
set_fontsize(16)

fig, ax = plt.subplots(
    figsize=(8, 6),
    subplot_kw={"projection": ccrs.PlateCarree(central_longitude=180)},
)

cloud_bands_masked = np.ma.masked_where(cloud_bands == 0, cloud_bands)
ax.contourf(
    lons, lats, cloud_bands_masked, transform=ccrs.PlateCarree(), cmap="gray_r"
)

for props in blobs_regionprops:
    centroid = props.centroid
    x_centroid, y_centroid = (
        lons[int(centroid[1])].item(),
        lats[int(centroid[0])].item(),
    )

    # Extract ellipse properties
    major_axis_length = props.major_axis_length
    minor_axis_length = props.minor_axis_length
    orientation = props.orientation
    angle_deg = (orientation * 360) / (2 * np.pi)

    ax.scatter(
        x_centroid,
        y_centroid,
        color="g",
        marker="o",
        s=40,
        transform=ccrs.PlateCarree(),
    )

    # Create and plot the ellipse with proper orientation
    ellipse = Ellipse(
        (x_centroid, y_centroid),
        width=major_axis_length * 0.5,
        height=minor_axis_length * 0.5,
        angle=angle_deg - 90,
        edgecolor="blue",
        facecolor="none",
        linewidth=2,
        transform=ccrs.PlateCarree(),
    )
    ax.add_patch(ellipse)

    # Plot major and minor axes using the provided code
    x0, y0 = x_centroid, y_centroid
    x1 = x0 + np.cos(orientation) * 0.25 * minor_axis_length
    y1 = y0 + np.sin(orientation) * 0.25 * minor_axis_length
    x2 = x0 - np.sin(orientation) * 0.25 * major_axis_length
    y2 = y0 + np.cos(orientation) * 0.25 * major_axis_length

    ax.plot((x0, x2), (y0, y2), "--r", linewidth=2.5, transform=ccrs.PlateCarree())
    ax.plot((x0, x1), (y0, y1), "--r", linewidth=2.5, transform=ccrs.PlateCarree())
    ax.text(
        x0 + 3,
        y0 - 2,
        str(int(angle_deg)) + r"°",
        color="k",
        transform=ccrs.PlateCarree(),
    )

lat_north, lat_south = 10, -45
lonticks = np.concatenate((np.arange(0, 180, 20), np.arange(-180, 0, 20)))
latticks = np.arange(-50, 20, 10)
ax.set_yticks(latticks, crs=ccrs.PlateCarree())
ax.set_xticks(lonticks, crs=ccrs.PlateCarree())
ax.xaxis.set_major_formatter(LON_FORMAT)
ax.yaxis.set_major_formatter(LAT_FORMAT)
ax.yaxis.set_minor_locator(MultipleLocator(5))
ax.xaxis.set_minor_locator(MultipleLocator(5))
ax.coastlines("50m", color="#404040")
ax.set_extent([wrapTo180(lons)[-1], wrapTo180(lons)[0], lat_south, lat_north])
ax.set_ylim([lat_south, lat_north])
original_startdate = config_event['startdate']
datetime_obj = dt.datetime.strptime(original_startdate, '%Y%m%d.%H')
formatted_startdate = datetime_obj.strftime('%Y-%m-%d')


plt.savefig(
    f"./ellipse_{formatted_startdate}.png",
    dpi=200,
    bbox_inches="tight",
)